In [ ]:
%matplotlib inline

# first time user needs obspy, it will be installed
try:
    import obspy
    from obspy.clients.fdsn import Client
except:
    !pip install obspy --user
    import obspy
    from obspy.clients.fdsn import Client

import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 5

# Python adaptation of lab_response.m by Carl Tape.
# Adapted from Lion Krischer
# Coding by Amanda McPherson, 2020
# Used in GEOS626
# Run this cell once at the beginning of each session

In [ ]:
# Point ObsPy at the desired client:
# ObsPy knows which website to connect to for many data centers.
# For others you can also pass the full URL.
c = Client("IRIS")

# Define time period for which you want data:
starttime = obspy.UTCDateTime(2004, 12, 26, 0, 58, 50)
# Create a new time object by adding 5 days to the previous ones. Time
# differences are in seconds.
endtime = starttime + 86400 * 5

# This will get all stations that satisfy all constraints simultaneosly.
inv = c.get_stations(network="G", station="CAN", channel="BHZ", 
                     level="response", 
                     starttime=starttime, endtime=endtime)

print(inv)
print(inv[0])
print(inv[0][0])
print(inv[0][0][0])
sps = 20.0     # change this to be accessed from in
fNy = 1/(2*sps)

In [ ]:
# Inspect the instrument response:
inv.plot_response(min_freq=0.0001);

In [ ]:
# See more details about the response:
print(inv[0][0][0].response)

In [ ]:
# Leave out the digitizer and FIR filters, and examine the "simple" pole-zero plot.
# Compare it with the plot above.
# TRY OUT: Replace VEL with ACC or DISP to see response for velocity or acceleration
inv[0][0][0].plot(0.0001, start_stage=1, end_stage=1, output="VEL");

In [ ]:
# now plot everything together for comparison

# define a set of frequencies to evaluate the response
f = np.logspace(-4.0, 2.0, num=100)
# get response functions
Id = inv[0][0][0].response.get_evalresp_response_for_frequencies(f,output='DISP')
Iv = inv[0][0][0].response.get_evalresp_response_for_frequencies(f,output='VEL')
Ia = inv[0][0][0].response.get_evalresp_response_for_frequencies(f,output='ACC')

fig, ax = plt.subplots(3,2,figsize=(14,16))
ax[0,0].semilogx(f,np.angle(Id)*180/np.pi, color='b')
ax[0,1].loglog(f,abs(Id), color='b')
ax[1,0].semilogx(f,np.angle(Iv)*180/np.pi, color='b')
ax[1,1].loglog(f,abs(Iv), color='b')
ax[2,0].semilogx(f,np.angle(Ia)*180/np.pi, color='b')
ax[2,1].loglog(f,abs(Ia), color='b')
#plt.subplots_adjust(hspace=0.5)
plt.show()

In [ ]:
# same plot but with more annotation

# max values
max_Id = max(Id)
f_id = f[np.nonzero(Id == max_Id)]
max_Iv = max(Iv)
f_iv = f[np.nonzero(Iv == max_Iv)]
max_Ia = max(Ia)
f_ia = f[np.nonzero(Ia == max_Ia)]

fig, ax = plt.subplots(3,2,figsize=(14,16))

ax[0,0].semilogx(f,np.angle(Id)*180/np.pi, color='b')
ax[0,0].set(xlabel='Frequency (Hz)', ylabel='Phase (deg)')
ax[0,0].axvline(fNy,c='r',lw=0.5,ls='--',dashes=[12,12])

ax[0,1].loglog(f,abs(Id), color='b')
ax[0,1].set(xlabel='Frequency (Hz)', ylabel='Amplitude')
ax[0,1].axvline(fNy,c='r',lw=0.5,ls='--',dashes=[12,12])
ax[0,1].set_title('(m to counts) max = %.2e at %.2e Hz'% (np.real(max_Id),f_id),fontsize=11)

ax[1,0].semilogx(f,np.angle(Iv)*180/np.pi, color='b')
ax[1,0].set(xlabel='Frequency (Hz)', ylabel='Phase (deg)')
ax[1,0].axvline(fNy,c='r',lw=0.5,ls='--',dashes=[12,12])

ax[1,1].loglog(f,abs(Iv), color='b')
ax[1,1].set(xlabel='Frequency (Hz)', ylabel='Amplitude')
ax[1,1].axvline(fNy,c='r',lw=0.5,ls='--',dashes=[12,12])
ax[1,1].set_title('(m/s to counts) max = %.2e at %.2e Hz'% (np.real(max_Iv),f_iv),fontsize=11)

ax[2,0].semilogx(f,np.angle(Ia)*180/np.pi, color='b')
ax[2,0].set(xlabel='Frequency (Hz)', ylabel='Phase (deg)')
ax[2,0].axvline(fNy,c='r',lw=0.5,ls='--',dashes=[12,12])

ax[2,1].loglog(f,abs(Ia), color='b')
ax[2,1].set(xlabel='Frequency (Hz)', ylabel='Amplitude')
ax[2,1].set_title('(m/s2 to counts) max = %.2e at %.2e Hz'% (np.real(max_Ia),f_ia),fontsize=11)
ax[2,1].axvline(fNy,c='r',lw=0.5,ls='--',dashes=[12,12])

plt.subplots_adjust(hspace=0.25)
plt.show()